# Food Delivery Analysis

### Step 1: Import Required Libraries

In [2]:
import pandas as pd
import sqlite3

### Step 2: Load CSV Data (orders.csv)

In [4]:
orders = pd.read_csv("orders.csv")

print("Orders Data Preview:")
orders.head()

Orders Data Preview:


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


### Step 3: Load JSON Data (users.json)

In [9]:
users = pd.read_json("users.json")

print("Users Data Preview:")
users.head()

Users Data Preview:


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


### Step 4: Load SQL Data (restaurants.sql)

##### 4.1 Create SQLite Database & Load SQL File

In [13]:
conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

##### 4.2 Read Restaurant Table into Pandas

In [15]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants Data Preview:")
restaurants.head()

Restaurants Data Preview:


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


### Step 5: Merge the Datasets

##### 5.1 Merge Orders + Users (LEFT JOIN)

In [20]:
merged_df = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

##### 5.2 Merge Result + Restaurants (LEFT JOIN)

In [23]:
final_df = pd.merge(
    merged_df,
    restaurants,
    how="left",
    on="restaurant_id"
)

### Step 6: Verify Final Dataset

In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [28]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


### Step 7: Save Final Dataset

In [32]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

### Step 8: Converting Order date into DataTime Format

In [35]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

In [37]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## A1. City with highest total revenue from Gold members

In [44]:
gold_city_revenue = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

## A2. Cuisine with highest average order value

In [47]:
final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

## A3. Distinct users with total spend > ₹1000

In [50]:
user_spend = final_df.groupby('user_id')['total_amount'].sum()
count_users = user_spend[user_spend > 1000].count()
count_users

2544

## A4. Rating range with highest total revenue

In [53]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)

final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_23712\3942643118.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

## A5. Gold members: city with highest average order value

In [56]:
(
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .mean()
    .sort_values(ascending=False)
)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

## A6. Cuisine with lowest number of distinct restaurants but good revenue

In [59]:
restaurant_count = final_df.groupby('cuisine')['restaurant_id'].nunique()
revenue = final_df.groupby('cuisine')['total_amount'].sum()

pd.concat([restaurant_count, revenue], axis=1)\
  .rename(columns={0:'restaurant_count',1:'revenue'})\
  .sort_values(by='restaurant_id')

,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


## A7. Percentage of orders placed by Gold members

In [65]:
gold_orders_pct = (
    final_df[final_df['membership'] == 'Gold'].shape[0]
    / final_df.shape[0]
) * 100

round(gold_orders_pct)

50

## A8. Restaurant with highest AOV but < 20 orders

In [78]:
restaurant_stats = final_df.groupby('restaurant_name_x').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

restaurant_stats[
    restaurant_stats['total_orders'] < 20
].sort_values(by='avg_order_value', ascending=False).head()

,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


## A9. Combination with highest revenue

In [74]:
final_df.groupby(['membership', 'cuisine'])['total_amount']\
        .sum()\
        .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

## A10. Quarter with highest total revenue

In [76]:
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

## B1. How many total orders were placed by users with Gold membership?

In [98]:
final_df[final_df['membership'] == 'Gold'].shape[0]

4987

## B2. Total revenue (rounded to nearest integer) from Hyderabad city

In [100]:
round(
    final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
)

1889367

## B3. How many distinct users placed at least one order?

In [102]:
final_df['user_id'].nunique()

2883

## B4. Average order value for Gold members (rounded to 2 decimals)

In [104]:
round(
    final_df[final_df['membership'] == 'Gold']['total_amount'].mean(),
    2
)

797.15

## B5. How many orders were placed for restaurants with rating ≥ 4.5?

In [106]:
final_df[final_df['rating'] >= 4.5].shape[0]

3374

## B6. How many orders were placed in the top revenue city among Gold members only?

### Step 1: Find top Gold city

In [108]:
top_gold_city = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

top_gold_city

'Chennai'

### Step 2: Count orders in that city (Gold only)

In [110]:
final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_gold_city)
].shape[0]

1337